## Introduction


In this lab, I will build an image concrete cruck classifier using the VGG16 pre-trained model, and will evaluate it and compare its performance to the model I built using the ResNet50 pre-trained model.

## Table of Contents

<div class="alert alert-block alert-info" style="margin-top: 20px">

<font size = 3>    

1. <a href="#item41">Download Data 
2. <a href="#item42">Part 1</a>
3. <a href="#item43">Part 2</a>  
4. <a href="#item44">Part 3</a>  

</font>
    
</div>

<a id="item41"></a>

## Download Data

Use the <code>wget</code> command to download the data for this assignment from here: https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DL0321EN/data/concrete_data_week4.zip

In [ ]:
!wget https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DL0321EN/data/concrete_data_week4.zip

--2021-03-04 08:17:16--  https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DL0321EN/data/concrete_data_week4.zip
Resolving s3-api.us-geo.objectstorage.softlayer.net (s3-api.us-geo.objectstorage.softlayer.net)... 67.228.254.196
Connecting to s3-api.us-geo.objectstorage.softlayer.net (s3-api.us-geo.objectstorage.softlayer.net)|67.228.254.196|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 261483817 (249M) [application/zip]
Saving to: ‘concrete_data_week4.zip’

concrete_data_week4 100%[===================>] 249.37M  31.0MB/s    in 7.6s    

2021-03-04 08:17:24 (32.7 MB/s) - ‘concrete_data_week4.zip’ saved [261483817/261483817]



In [ ]:
!unzip concrete_data_week4.zip

<a id="item42"></a>

## Part 1

In [ ]:
import keras
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense
from keras.applications import VGG16
from keras.applications.vgg16 import preprocess_input

In [ ]:
# DEFINE GLOBAL CONSTANTS
num_classes = 2

image_resize = 224
batch_size_train = 128
batch_size_valid = 128

In [ ]:
# CONSTRUCT AN ImageDataGenerator
data_generator = ImageDataGenerator(preprocessing_function = preprocess_input)


# GETTING TRAINING IMAGES
train_generator = data_generator.flow_from_directory(
                                                     "concrete_data_week4/train",
                                                      target_size = (image_resize, image_resize),
                                                      batch_size = batch_size_train,
                                                      class_mode = "categorical")

# GETTING VALIDATION IMAGES
valid_generator = data_generator.flow_from_directory(
                                                     "concrete_data_week4/valid",
                                                      target_size = (image_resize, image_resize),
                                                      batch_size = batch_size_valid,
                                                      class_mode = "categorical")

Found 30001 images belonging to 2 classes.
Found 9501 images belonging to 2 classes.


In [ ]:
# CREATING SEQUENTIAL MODEL
model = Sequential()

In [ ]:
model.add(VGG16(include_top = False, pooling = "avg", weights = "imagenet"))

In [ ]:
model.add(Dense(num_classes, activation = "softmax"))

In [ ]:
# JUST FOR INFORMATION ABOUT CONSTRUCTION OF VGG16 PRE-TRAINED 
#model.layers
model.layers[0].layers

In [ ]:
model.layers[0].trainable = False
model.summary()

In [ ]:
model.compile(optimizer = "adam", loss = "categorical_crossentropy", metrics = ["accuracy"])

In [ ]:
steps_per_epoch_train = len(train_generator)
steps_per_epoch_valid = len(valid_generator)

num_epochs = 2

In [ ]:
history = model.fit_generator(train_generator,
                   steps_per_epoch = steps_per_epoch_train,
                   epochs = num_epochs,
                   validation_data = valid_generator,
                   validation_steps = steps_per_epoch_valid,
                   verbose = 1)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/2
235/235 [==============================] - 234s 782ms/step - loss: 0.6008 - accuracy: 0.8483 - val_loss: 0.0496 - val_accuracy: 0.9877
Epoch 2/2
235/235 [==============================] - 172s 733ms/step - loss: 0.0419 - accuracy: 0.9911 - val_loss: 0.0287 - val_accuracy: 0.9940


In [ ]:
model.save("vgg16_cocnrete_classifier_model.h5")

In [ ]:
from google.colab import files


<a id="item43"></a>

## Part 2

In this part, I will evaluate deep learning models on a test data.


In [ ]:
# model_resnet50 = files.upload()

In [ ]:
# model_vgg16 = files.upload()

In [ ]:
test_generator = data_generator.flow_from_directory(
                                                     "concrete_data_week4/test",
                                                      target_size = (image_resize, image_resize),
                                                      shuffle = False)

Found 500 images belonging to 2 classes.


In [ ]:
from keras.models import load_model

In [ ]:
model_resnet = load_model("classifier_resnet_model.h5")

In [ ]:
model_vgg = load_model("vgg16_cocnrete_classifier_model.h5")


In [ ]:
model_resnet.evaluate_generator(test_generator)

In [ ]:
model_vgg.evaluate_generator(test_generator)

<a id="item44"></a>

## Part 3

In this model, I will predict whether the images in the test data are images of cracked concrete or not.


In [ ]:
resnet_predict = model_resnet.predict_generator(test_generator)

In [ ]:
vgg16_predict = model_vgg.predict_generator(test_generator)

In [ ]:
 # label_map = (test_generator.class_indices)
 # vgg16_predict

In [ ]:
for i in range(0,500,100):
  if vgg16_predict[i][0] > 0.5:
    print("Negative")
  else:
    print("Positive")

# vgg16_predict[345][0]

In [ ]:
for i in range(0,500,100):
  if resnet_predict[i][0] > 0.5:
    print("Negative")
  else:
    print("Positive")
